## Comparing local oscillators, original vs. HM's implementation

Original first, show frequency responses

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# spectrome imports
from spectrome.brain import Brain
from spectrome.utils import functions, path
from spectrome.forward import eigenmode, runforward

In [2]:
# House keeping cell:

# directory for HCP template connectomes
hcp_dir = './../../data'

# define frequency range of interest
fmin = 2
fmax = 45
fvec = np.linspace(fmin, fmax, 40)

# Initialize Brain object
HCP_brain = Brain.Brain()
HCP_brain.add_connectome(hcp_dir)
HCP_brain.reorder_connectome(HCP_brain.connectome, HCP_brain.distance_matrix)
HCP_brain.bi_symmetric_c()
HCP_brain.reduce_extreme_dir()